In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.layers import Input, Dense, Conv1D, Flatten, MaxPooling1D
from keras.models import Model, Sequential
from keras.layers.merge import concatenate

Using TensorFlow backend.


#### This Time series model example devided into 4 parts
<ol>
    <li>Univariate CNN Models</li>
    <li>Multivariate CNN Models</li>
    <li>Multi-Step CNN Models</li>
    <li>Multivariate Multi-Step CNN Models</li>
</ol>

### 1.Univariate CNN Models

In [2]:
data = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140]

divide the sequence into multiple input/output patterns called samples, 
where three time steps are used as input and one time step is used as 
output for the one-step prediction that is being learned.

In [3]:
input_dim = 3
n_feature = 1
n = len(data)
assert (input_dim + n_feature) < n
def split_sequence(data, input_dim):
    feature = []
    label = []
    start = 0
    stop = start + input_dim
    while stop < n:
        feature.append(data[start:stop])
        label.append(data[stop])
        start += 1
        stop += 1
    return np.array(feature), np.array(label)

In [4]:
X, y = split_sequence(data, input_dim)
X.shape

(11, 3)

In [5]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
X = X.reshape(X.shape[0], X.shape[1], n_feature)
X[0].shape

(3, 1)

In [6]:
# creating model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(input_dim, n_feature)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(units=50, activation='relu'))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=1000, verbose=0)

In [7]:
x_input = np.array([120, 130, 140])
x_input = x_input.reshape((1, input_dim, n_feature))
yhat = model.predict(x_input, verbose=5)
print(yhat)

[[154.837]]


In [8]:
X[0]

array([[10],
       [20],
       [30]])

### 2. Multivariate CNN models

It has 2 main type:
    1. multiple input series
    2. Multiple Parallel Series

### 1.multiple input series:
A problem may have two or more parallel input time series and an output time series that is dependent on the input time series.

In [9]:
feature_1 = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
feature_2 = np.array([15, 25, 35, 45, 55, 65, 75, 85, 95, 105])
label = feature_1 + feature_2
dataset = np.concatenate([feature_1[:, np.newaxis], feature_2[:, np.newaxis], label[:, np.newaxis]], axis = 1)

In [10]:
print(dataset)
time_step = 3
n_feature = 2

[[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]
 [100 105 205]]


In [11]:
def split_data(data, time_step, feature_column, label_column):
    n = len(data)
    features = []
    labels = []
    for start in range(n-time_step+1):
        feature = data[start: start+time_step , feature_column].tolist()
        label = data[start+time_step-1, label_column].tolist()
        features.append(feature)
        labels.append(label)
    return np.array(features), np.array(labels)
X, y = split_data(dataset, time_step, [0, 1], 2)

In [12]:
# model creation
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape= (time_step, n_feature)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [13]:
model.fit(X, y, epochs=100, verbose=0)

In [14]:
test = np.array([[90, 95], [100, 105], [110, 115]])
#test = np.array([[80, 85], [ 90,  95], [100, 105]])
test = test.reshape((1, time_step, n_feature))
model.predict(test)

array([[228.87364]], dtype=float32)

Alternatively Each input series can be handled by a separate CNN and the output of
each of these submodels can be combined before a prediction is made for the output
sequence.We can refer to this as a multi-headed CNN model. It may offer more flexibility
or better performance depending on the specifics of the problem that is being modeled.
For example, it allows you to configure each sub-model differently for each input series,
such as the number of filter maps and the kernel size.

we can define the first input model as a 1D CNN with an input layer that expects vectors with n_steps and 1 feature.

In [15]:
# first input model
visible1 = Input(shape=(time_step, 1))
cnn1 = Conv1D(filters=64, kernel_size=2, activation='relu')(visible1)
cnn1 = MaxPooling1D(pool_size=2)(cnn1)
cnn1 = Flatten()(cnn1)

In [16]:
# second input model
visible2 = Input(shape=(time_step, 1))
cnn2 = Conv1D(filters=64, kernel_size=2, activation='relu')(visible2)
cnn2 = MaxPooling1D(pool_size=2)(cnn2)
cnn2 = Flatten()(cnn2)

In [17]:
merge = concatenate([cnn1, cnn2])
dense = Dense(units=50, activation='relu')(merge)
output = Dense(units=1)(dense)
model = Model(inputs=[visible1, visible2], outputs=output)

In [18]:
X1, y = split_data(dataset, time_step, [0], 2)
X2, y = split_data(dataset, time_step, [1], 2)

In [19]:
model.compile(optimizer='adam', loss='mse')

In [20]:
model.fit([X1, X2], y, epochs=100, verbose=0)

In [21]:
X1 = test[:, :, [0]]
X2 = test[:, :, [1]]
y_pred = model.predict([X1, X2], verbose=0)

In [22]:
y_pred

array([[230.00555]], dtype=float32)

### 2.Multiple Parallel series

An alternate time series problem is the case where there are
multiple parallel time series and a value must be predicted for each.

In [23]:
feature_1 = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
feature_2 = np.array([15, 25, 35, 45, 55, 65, 75, 85, 95, 105])
feature_3 = feature_1 + feature_2

In [24]:
def split_data_multiple_parallel(data, time_step, feature_column, label_column):
    n = len(data)
    features = []
    labels = []
    for start in range(n-time_step):
        feature = data[start: start+time_step , feature_column].tolist()
        label = data[start+time_step, label_column].tolist()
        features.append(feature)
        labels.append(label)
    return np.array(features), np.array(labels)
dataset = np.concatenate([feature_1[:, np.newaxis], feature_2[:, np.newaxis], feature_3[:, np.newaxis]], axis = 1)
time_step = 3
X, y = split_data_multiple_parallel(dataset, time_step, [0, 1, 2], [0, 1, 2])
n_feature = X.shape[2]

In [25]:
# define models
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(time_step, n_feature)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(units=50, activation='relu'))
model.add(Dense(n_feature))
model.compile(optimizer='adam', loss='mse')

In [26]:
model.fit(X, y, epochs=1000, verbose=False)

In [27]:
test = np.array([[80,  85, 165], [ 90,  95, 185], [100, 105, 205]])
test = test.reshape(1, test.shape[0], test.shape[1])
model.predict(test, verbose=0)

array([[111.43784, 116.61249, 228.50899]], dtype=float32)

### 3.  multistep CNN models

In [28]:
feature = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
time_step = 3
output_size = 2

In [29]:
def split_seq_multistep(data, time_step, output_size):
    feature = []
    label = []
    for start in range(len(data)-time_step-output_size):
        feature.append(data[start: start+ time_step])
        label.append(data[start+ time_step: start+ time_step + output_size])
    return np.array(feature), np.array(label)

In [30]:
X, y = split_seq_multistep(feature, time_step, output_size)

In [31]:
# reshape from [n_sample, timestep, n_feature]
n_feature = 1
X = X.reshape((X.shape[0], X.shape[1], n_feature))

In [32]:
n_step_out = 2
# defining model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X.shape[1], n_feature)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(units=50, activation='relu'))
model.add(Dense(units=n_step_out))
model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=1000, verbose=0)

In [33]:
test = np.array([[80, 90, 100]])
test = test.reshape((1, test.shape[1], n_feature))

In [34]:
model.predict(test, verbose=0)

array([[112.72062 , 125.181526]], dtype=float32)

### 4.Multivariate Multi-Step CNN Models

In [35]:
feature1 = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
feature1 = np.array([15, 25, 35, 45, 55, 65, 75, 85, 95, 105])
label = feature_1 + feature_2

In [36]:
data = np.concatenate([feature_1[:, np.newaxis], feature_2[:, np.newaxis], label[:, np.newaxis]], axis=1)

In [58]:
def split_multivariate_multistep_data(data, n_step_in, n_step_out):
    n = len(data)
    feature = []
    label = []
    for start in range(n-n_step_in-n_step_out+1):
        feature.append(np.squeeze(data[start:start+n_step_in]))
        label.append(np.squeeze(data[start+n_step_in:start+n_step_in+n_step_out]))
    return np.array(feature) , np.array(label)
n_step_in, n_step_out = 3, 2
X, y = split_multivariate_multistep_data(data, n_step_in, n_step_out)

In [89]:
print(X)
#print(y)

[[[ 10  15  25]
  [ 20  25  45]
  [ 30  35  65]]

 [[ 20  25  45]
  [ 30  35  65]
  [ 40  45  85]]

 [[ 30  35  65]
  [ 40  45  85]
  [ 50  55 105]]

 [[ 40  45  85]
  [ 50  55 105]
  [ 60  65 125]]

 [[ 50  55 105]
  [ 60  65 125]
  [ 70  75 145]]

 [[ 60  65 125]
  [ 70  75 145]
  [ 80  85 165]]]


In [88]:
n_feature = X.shape[2]
n_output = y.shape[1]*y.shape[2]
# model creation
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_step_in, n_feature)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(units=50, activation='relu'))
model.add(Dense(units=n_output))
model.compile(optimizer='adam', loss='mse')
model.fit(X, y.reshape((y.shape[0], y.shape[1] * y.shape[2])), epochs=8000, verbose=0)

In [100]:
test = np.array([[ 80, 85, 165],[90, 95, 185], [100, 105, 205]])
predict = model.predict(test.reshape(1, test.shape[0], test.shape[1]), verbose=0)

In [101]:
predict.reshape(n_step_out, n_feature)

array([[111.905685, 116.928825, 229.11    ],
       [123.33852 , 129.6891  , 252.42561 ]], dtype=float32)

In [102]:
data

array([[ 10,  15,  25],
       [ 20,  25,  45],
       [ 30,  35,  65],
       [ 40,  45,  85],
       [ 50,  55, 105],
       [ 60,  65, 125],
       [ 70,  75, 145],
       [ 80,  85, 165],
       [ 90,  95, 185],
       [100, 105, 205]])